---
title: 'Recurrent Neural Networks'
jupyter: python3
bibliography: references.bib
---

# RNN Theory

In [ ]:
#| echo: false
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from IPython.display import Image

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/DS701-Course-Notes/blob/main/ds701_book/jupyter_notebooks/27-RNN.ipynb)

We introduce recurrent neural networks (RNNs) which is a neural network architecture used for machine learning on sequential data.

## What are RNNs?

:::: {.columns}
::: {.column width="50%"}

- A type of artificial neural network designed for processing sequences of data.
- Unlike traditional neural networks, RNNs have connections that form directed cycles, allowing information to persist.
:::
::: {.column width="50%"}
![](drawio/RNN.png)
:::
::::

The above figure shows an RNN architecture. The block $A$ can be viewed as a chunk of the RNN. $A$ accepts as input $x_t$ and outputs a value $\hat{y}_t$. The loop with the hidden state $h_{t-1}$ illustrates how information passes from one step of the network to the next.

## Why Use RNNs?

- **Sequential Data**: Ideal for tasks where data points are dependent on previous ones.
- **Memory**: Capable of retaining information from previous inputs, making them powerful for tasks involving context. This is achieved from hidden states and feedback loops.

## RNN Applications

- **Natural Language Processing (NLP)**: Language translation, sentiment analysis, and text generation.
- **Speech Recognition**: Converting spoken language into text.
- **Time Series Prediction**: Forecasting stock prices, weather, and other temporal data.
- **Music Generation**: Creating a sequence of musical notes.

## Outline

1. Basic RNN Architecture
1. LSTM (Long Short-Term Memory) and GRU (Gated Recurrent Units)
1. Examples

## RNN Architecture

![](drawio/RNN-Full.png)

## Forward Propagation

The forward pass in the RNN architecture is given by the following operations

1. $h_t = g_h(W_{hh} h_{t-1} + W_{hx} x_t + b_h)$
1. $\hat{y}_t = g_y(W_{yh} h_t + b_y)$

The vector $x_t$ is the $t$-th element of the input sequence. The vector $h_t$ is the hidden state at the $t$-th point of the sequence. The dimension of $h_t$ is a hyperparameter of the RNN. 

The vector $\hat{y}_t$ is the $t$-th output of the sequence. The functions $g_h$ and $g_y$ are nonlinear activation functions. 

The weight matrices $W_{hh}$, $W_{hx}$, $W_{yh}$, and biases $b_h$, $b_y$ are trainable parameters **that are reused in each time step**. Note that we must define the vector $h_0$. A common choice is $h_0 = 0$.

## Model Types

Depending on the application there many be varying number of outputs $\hat{y}_t$. The figure below illustrates how the architecture can change.

:::: {.columns}
::: {.column width="60%"}
![](figs/RNNConfigurations.png)
:::
::: {.column width="40%"}
1. One-to-one, is a regular feed forward neural network.
1. One-to-many, e.g., image captioning (1 image and output a sequence of words).
1. Many-to-one, e.g., sentiment analysis from a sequence of words or stock price prediction.
1. Many-to-many, e.g., machine translation or video frame-by-frame action classification.
:::
::::

Our subsequent illustrations of RNNs all display many-to-many architectures.

## Stacked RNNs

It is also possible to stack multiple RNNs on top of each other. This is illustrated in the figure below.

![](drawio/StackedRNN.png)

## Loss Calculation for Sequential Data

For each prediction in the sequence $\hat{y}_t$ and corresponding true value $y_t$  we calculate the the cross-entropy loss $\mathcal{L}_t(y_t, \hat{y}_t) = -y_t\log(\hat{y}_t)$. The total loss is the sum of all the individual losses, $\mathcal{L} = \sum_t \mathcal{L}_t$. This is illustrated in the figure below.

If we were predicting continuous value we would use a mean-square error loss.

![](drawio/RNN-Loss.png)


## Vanishing Gradients

The key idea of RNNs is that earlier items in the sequence  influence the more recent outputs. This is highlighted in blue in the figure below.

However, for longer sequences the influence can be significantly reduced. This is illustrated in red in the figure below.

Due to the potential for long sequences of data, during training you are likely to encounter small gradients. As a result, RNNs are prone to vanishing gradients.

![](drawio/VanishingGradient.png)

## RNN Limitations

- **Vanishing Gradients**:
  - During training, gradients can become very small (vanish), making it difficult for the network to learn long-term dependencies.

- **Long-Term Dependencies**:
  - RNNs struggle to capture dependencies that are far apart in the sequence, leading to poor performance on tasks requiring long-term memory.

- **Difficulty in Parallelization**:
  - The sequential processing of data in RNNs makes it challenging to parallelize training, leading to slower training times.

- **Limited Context**:
  - Standard RNNs have a limited ability to remember context over long sequences, which can affect their performance on complex tasks.

- **Bottleneck Problem**:
  - The hidden state is all that carries forward history and it can be a bottleneck in how expressive it can be.    

## Addressing RNN Limitations

Some proposed solutions for mitigating the previous issues are

- **LSTM (Long Short-Term Memory)**
- **GRU (Gated Recurrent Units)** 

These are more advanced variants of RNNs designed to address some of these limitations by improving the ability to capture long-term dependencies and addressing gradient issues.

## LSTM

Below is an illustration of the LSTM architecture.

![](drawio/LSTM.png)

---

The new components are:

- The input cell state $c_{t-1}$ and output cell state $c_{t}$.
- Yellow blocks consisting of neural network layers with sigmoid $\sigma$ or tanh activation functions.
- Red circles indicating point-wise operations.


## Cell State

:::: {.columns}
::: {.column width="50%"}

The cell state $c_{t-1}$ is the input to the LSTM block.

This value then moves through the LSTM block.

It is modified by either a multiplication or addition interaction.

After these operations, the modified cell state is $c_{t}$ is sent to the next LSTM block. In addition, $c_t$ is added to the hidden state after tanh activation.

:::
::: {.column width="50%"}
![](drawio/CellState.png)
:::
::::

## Forget Gate Layer

:::: {.columns}
::: {.column width="50%"}
The forget layer computes a value 

$$
f_t = \sigma(W_f[h_{t-1}, x_t] + b_f),
$$

where $\sigma(x) = \frac{1}{1 + e^{-x}}$ is the sigmoid function.

The output $f_t$ is a vector of numbers between 0 and 1. These values multiply the corresponding vector values in $c_{t-1}$.

A value of 0 says throw that component of $c_{t-1}$ away. A value of 1 says keep that component of $c_{t-1}$.

This operation tells us which *old* information in the cell state we should keep or remove.

:::
::: {.column width="50%"}
![](drawio/ForgetLayer.png)
:::
::::

## Input Gate Layer

:::: {.columns}
::: {.column width="50%"}
The input gate layer computes values

$$
\begin{align}
i_t &= \sigma(W_i[h_{t-1}, x_t] + b_i), \\
\tilde{c}_{t} &= \operatorname{tanh}(W_c[h_t-1, x_t] + b_c).
\end{align}
$$

The value that is added to the cell state $c_{t-1}$ is $i_t \cdot \tilde{c}_t$.

This value tells us what *new* information to add to the cell state.

At this stage of the process, the cell state now has the formula

$$
c_t = f_t \cdot c_{t-1} + i_t \cdot \tilde{c}_t.
$$

:::
::: {.column width="50%"}
![](drawio/InputGateLayer.png)
:::
::::

## Output Layer

:::: {.columns}
::: {.column width="50%"}
The output gate layer computes values

$$
\begin{align}
o_t &= \sigma(W_o[h_{t-1}, x_t] + b_o), \\
h_{t} &=o_t * \operatorname{tanh}(c_t).
\end{align}
$$

The vector $o_t$ from the sigmoid layer tells us what parts of the cell state we use for output. The output is a filtered version of the hyperbolic tangent of cell state $c_t$. 

The output of the block is $\hat{y}_t$. It is the same as the hidden state $h_t$ that is sent to the $t+1$ LSTM block.

:::
::: {.column width="50%"}
![](drawio/OutputLayer.png)
:::
::::

## Weight Summary

For the LSTM architecture, we have the following sets of weights

- $W_f$ and $b_f$ (forget layer),
- $W_i, W_c$ and $b_i, b_c$ (input gate layer),
- $W_o$ and $b_0$ (output layer).


## Advantages of LSTMs

- **Long-term Dependencies**: LSTMs can capture long-term dependencies in sequential data, making them effective for tasks like language modeling and time series prediction.
- **Avoiding Vanishing Gradient**: The architecture of LSTMs helps mitigate the vanishing gradient problem, which is common in traditional RNNs.
- **Flexibility**: LSTMs can handle variable-length sequences and are versatile for different types of sequential data.
- **Memory**: They have a memory cell that can maintain information over long periods, which is crucial for tasks requiring context retention.

## Disadvantages of LSTMs

- **Complexity**: LSTMs are more complex than simpler models like traditional RNNs, leading to longer training times and higher computational costs.
- **Overfitting**: Due to their complexity, LSTMs are prone to overfitting, especially with small datasets.
- **Resource Intensive**: They require more computational resources and memory, which can be a limitation for large-scale applications.
- **Hyperparameter Tuning**: LSTMs have many hyperparameters that need careful tuning, which can be time-consuming and challenging.

## GRU

A variant of the LSTM architecture is the Gated Recurrence Unit.

:::: {.columns}
::: {.column width="50%"}
It combines the forget and input gates into a single update gate. It also combines the cell-state and hidden state. The operations that are now performed are given below:

$$
\begin{align}
z_t &= \sigma(W_z[h_{t-1}, x_t] + b_z),\\
r_t &= \sigma(W_r[h_{t-1}, x_t] + b_r), \\
\tilde{h}_t &= \sigma(W_{\tilde{h}}[h_{t-1}, x_t] + b_{\tilde{h}}), \\
h_t &= (1- z_t)\cdot h_{t-1} + z_t\cdot \tilde{h}_t.
\end{align}
$$
:::
::: {.column width="50%"}
![](drawio/GRU.png)
:::
::::

## Advantages of GRUs

- **Simpler Architecture**: GRUs have a simpler architecture compared to LSTMs, with fewer gates, making them easier to implement and train.
- **Faster Training**: Due to their simpler structure, GRUs often train faster and require less computational power than LSTMs.
- **Effective Performance**: GRUs can perform comparably to LSTMs on many tasks, especially when dealing with shorter sequences.
- **Less Prone to Overfitting**: With fewer parameters, GRUs are generally less prone to overfitting compared to LSTMs.

## Disadvantages of GRUs

- **Less Expressive Power**: The simpler architecture of GRUs might not capture complex patterns in data as effectively as LSTMs.
- **Limited Long-term Dependencies**: GRUs might struggle with very long-term dependencies in sequential data compared to LSTMs.
- **Less Flexibility**: GRUs offer less flexibility in terms of controlling the flow of information through the network.

# RNN Examples

## Appliance Energy Prediction

In this example we will use a [dataset](https://archive.ics.uci.edu/dataset/374/appliances+energy+prediction) containing the energy usage in Watt hours (Wh) of appliances in a low energy building.

In addition to the appliance energy information, the dataset includes the house temperature and humidity conditions. We will build an LSTM model that predicts the energy usage of the appliances.

## Load the Data

The code cells below load the dataset.

In [ ]:
#| code-fold: false
import os

file_path = 'energydata_complete.csv'
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv"

if os.path.exists(file_path):
    data = pd.read_csv(file_path)
else:
    data = pd.read_csv(url)
    data.to_csv(file_path, index=False)

data.head()

## Resample Data

We're interested in the `Appliances` column, which is the energy use of the appliances in Wh. 

First, we'll resample the data to hourly resolution and fill missing values using the forward fill method.

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

data = data['Appliances'].resample('h').mean().ffill() # Resample and fill missing

data.head()

## Preparing the Data

We create train-test splits and scale the data accordingly. 

In addition, we create our own dataset class. In this class we create lagged sequences of the energy usage data. The amount of lag, or look-back sequence length, is set to 24.

In [ ]:
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size

train_data = data[:train_size]
test_data = data[train_size:]

# Normalize data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
test_data_scaled = scaler.transform(test_data.values.reshape(-1, 1))

# Prepare data for LSTM
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, index):
        X = self.data[index:index + self.seq_length]
        y = self.data[index + self.seq_length]
        return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

seq_length = 24
train_dataset = TimeSeriesDataset(train_data_scaled, seq_length)
test_dataset = TimeSeriesDataset(test_data_scaled, seq_length)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

## LSTM Architecture

We define our LSTM architecture in the code cell below.

In [ ]:
#| code-fold: false
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=16, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Use the output of the last time step
        return x

model = LSTMModel()
print(model)

<!-- 
## LSTM Architecture Visualized

```{.python}
from torchview import draw_graph
# Create a dummy input tensor with the shape (batch_size, sequence_length, input_size)
dummy_input = torch.randn(1, 10, 1)

# Perform a forward pass to create the computation graph
output = model(dummy_input)

# Generate and save the visualization of the model
# Generate the visualization of the model
model_graph = draw_graph(model, input_size=(1, 10, 1), depth=2, hide_inner_tensors=True)

# Display the graph
model_graph.visual_graph

Image(filename='LSTMModel.png')
``` 
-->

## Loss and Training

We use a mean-square error loss function and the Adams optimizer. We train the model for 20 epochs. We display the decrease in the loss during training. Based on the plot, did the optimizer converge?

In [ ]:
# Set criterion and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 20
train_losses = []
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for X, y in train_loader:

      optimizer.zero_grad()
      outputs = model(X)
      loss = criterion(outputs, y)
      loss.backward()
      optimizer.step()

      train_loss += loss.item()
    train_losses.append(train_loss)

# Plot the training losses over epochs
plt.figure(figsize=(10, 5))
plt.plot(range(1, epochs + 1), train_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.xticks(range(1, epochs + 1))  # Set x-tick marks to be integers
plt.grid(True)
plt.show()

## Evaluation

We'll evaluate the model's performance by plotting the predicted values from the test set on top of the actual test set values. 

How did the model perform? Are there any obvious problems?

In [ ]:
# Evaluate the model
model.eval()
predictions = []
trues = []
with torch.no_grad():
    for X, y in test_loader:
        preds = model(X)
        predictions.extend(preds.numpy())
        trues.extend(y.numpy())

# Rescale predictions and true to original scale
predictions_rescaled = scaler.inverse_transform(predictions)
true_rescaled = scaler.inverse_transform(trues)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(true_rescaled, label='True Values')
plt.plot(predictions_rescaled, label='Predicted Values', alpha=0.7)
plt.legend()
plt.show()

<!-- 
## Music Generation

The goal is to create an LSTM architecture to create a piece of music. We will train the LSTM on some example snippets of music. The trained model will then be able to generate a new piece of music.

We will first need to write some helper functions to get this to work.

## MIDI Data

We will use the package [pretty_midi](https://github.com/craffel/pretty-midi). This package will allow us to convert MIDI (Musical Instrument Digital Interface) files into NumPy arrays. MIDI files are a common format for music data. They contain information about the notes, timing, and instruments used in a piece of music. We can then use the NumPy arrays to train our LSTM.

Below is the code that will convert our MIDI file to a NumPy array. The output is an array of the start time of the note, the end time of the note, the note pitch, and the note velocity.

```{.python}
#| code-fold: false
import pretty_midi
import numpy as np

def midi_to_notes(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    notes = []
    for instrument in midi_data.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append([note.start, note.end, note.pitch, note.velocity])
    return np.array(notes)
```

## Maestro dataset

The [Maestro](https://magenta.tensorflow.org/datasets/maestro#v300) dataset is a dataset composed of about 200 hours of virtuosic piano performances. I took a few small samples from some of these files to train our LSTM and generate some new music.

```{.python}
#| code-fold: false
midi_files = [
    os.path.join('music', 'music_sample_1.mid'),
    os.path.join('music', 'music_sample_2.mid'),
    os.path.join('music', 'music_sample_3.mid')]
all_notes = []
for midi_file in midi_files:
  notes = midi_to_notes(midi_file)
  all_notes.append(notes)
all_notes = np.concatenate(all_notes)
```

```{.python}
import subprocess

def midi_to_mp3(midi_file, mp3_file):
    # Use the 'timidity' tool to convert MIDI to WAV
    wav_file = midi_file.replace('.mid', '.wav')
    subprocess.run(['timidity', midi_file, '-Ow', '-o', wav_file])
    
    # Use the 'ffmpeg' tool to convert WAV to MP3
    subprocess.run(['ffmpeg', '-i', wav_file, mp3_file])

def midi_to_wav(midi_file):
    # Use the 'timidity' tool to convert MIDI to WAV
    wav_file = midi_file.replace('.mid', '.wav')
    print(wav_file)
    print(f"Converting {midi_file} to {wav_file}")
    subprocess.run(['timidity', midi_file, '-Ow', '-o', wav_file])
    

# Convert the generated MIDI file to MP3
#midi_to_mp3('music/music_sample_1.mid', 'music/music_sample_1.mp3')
#midi_to_mp3('music/music_sample_2.mid', 'music/music_sample_2.mp3')
#midi_to_mp3('music/music_sample_3.mid', 'music/music_sample_3.mp3')
print(os.path.exists(os.path.join('music', 'music_sample_1.mid')))
if not os.path.exists(os.path.join('music', 'music_sample_1.wav')):
    midi_to_wav(os.path.join('music', 'music_sample_1.mid'))
if not os.path.exists(os.path.join('music', 'music_sample_2.wav')):
    midi_to_wav(os.path.join('music', 'music_sample_2.mid'))
if not os.path.exists(os.path.join('music', 'music_sample_3.wav')):
    midi_to_wav(os.path.join('music', 'music_sample_3.mid'))
```

<audio src="music/music_sample_1.wav" controls></audio>
<audio src="music/music_sample_2.wav" controls></audio>
<audio src="music/music_sample_3.wav" controls></audio>


## Sequences of Notes

We will convert the note sequences into a format suitable for training the LSTM model. An example of the split is shown below.

![](drawio/TrainTestPrep.png)


This means we create sequences of notes and corresponding targets.


```{.python}
#| code-fold: false
sequence_length = 48  # Length of each input sequence
X = []
y = []

for i in range(len(all_notes) - sequence_length):
    X.append(all_notes[i:i+sequence_length])
    y.append(all_notes[i+sequence_length])

X = np.array(X)
y = np.array(y)
print("Shape of X", X.shape)
print("Shape of y", y.shape)
```

## LSTM Architecture

We now create the LSTM architecture.


```{.python}
#| code-fold: false
import torch
import torch.nn as nn

class MusicLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(MusicLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Example usage
input_dim = X.shape[2]  # Number of features (e.g., pitch, velocity)
hidden_dim = 20
num_layers = 2
output_dim = X.shape[2]
model = MusicLSTM(input_dim, hidden_dim, num_layers, output_dim)
```


## Training

We will train our model for 100 epochs.

```{.python}
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
batch_size = 8
train_losses = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for i in range(0, len(X), batch_size):
        inputs = torch.tensor(X[i:i+batch_size], dtype=torch.float32)
        targets = torch.tensor(y[i:i+batch_size], dtype=torch.float32)
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_losses.append(train_loss)
# Plot the training losses over epochs
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, marker='o', linestyle='-', color='b')
plt.title('Training Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()
```

## Generate

We next write a function that will help us generate a new piece of music.

```{.python}
#| code-fold: false
def generate_music(model, start_sequence, length):
    model.eval()
    generated = start_sequence
    for _ in range(length):
        input_seq = torch.tensor(generated[-sequence_length:], dtype=torch.float32).unsqueeze(0)
        next_note = model(input_seq).detach().numpy()
        generated = np.vstack((generated, next_note))
    return generated

# Example usage
start_sequence = X[10]  # Starting sequence for generation
generated_music = generate_music(model, start_sequence, 100)  # Generate 100 notes
print(generated_music.shape)
```

## Download the Song

We can generate an output midi file to play. The generated song is not very good. We could probably create a better song through an improved model, more training, and a larger training set.

```{.python}
def notes_to_midi(notes, output_file):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    for note in notes:
        start, end, pitch, velocity = note
        midi_note = pretty_midi.Note(
            velocity=int(velocity),
            pitch=int(pitch),
            start=float(start),
            end=float(end)
        )
        instrument.notes.append(midi_note)
    midi.instruments.append(instrument)
    midi.write(output_file)

# Uncomment this line to write the following generated music to a MIDI file
notes_to_midi(generated_music, os.path.join('music', 'generated_music.midi'))
```

## Play the Generated Song

Now let's play the generated song.

```{.python}
import subprocess

def midi_to_wav(midi_file):
    # Use the 'timidity' tool to convert MIDI to WAV
    wav_file = midi_file.replace('.midi', '.wav')
    print(f"Converting {midi_file} to {wav_file}")
    subprocess.run(['timidity', midi_file, '-Ow', '-o', wav_file])
 
if not os.path.exists(os.path.join('music', 'generated_music.wav')):
    midi_to_wav(os.path.join('music', 'generated_music.midi'))
```

<audio src="music/generated_music.wav" controls></audio> 
-->

## Other Applications

Andrej Karpathy has an excellent blog post titled [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/). We will describe some of the applications below.

## Shakespeare

:::: {.columns}
::: {.column width="50%"}
Using the a 3-layer RNN with 512 hidden nodes on each layer, Karpathy trained a model on the complete works of Shakespeare. With this model he was able to generate texts similar to what is seen on the left.

Observe that there are some typos and the syntax is not perfect, but overall the style appears Shakespearean.

:::
::: {.column width="5-%"}
![](figs/ShakespeareText.png)
:::
::::

## Wikipedia

:::: {.columns}
::: {.column width="30%"}
Using a 100MB [dataset](http://prize.hutter1.net) of raw Wikipedia data, Karpathy trained RNNs which generated Wikipedia content. An example of the generated markdown is shown to the right.

The text is nonsense, but is structured like a Wikipedia post.
Interestingly, the model *hallucinated* and fabricated a url that does not exist.
:::
::: {.column width="70%"}
![](figs/WikipediaContent.png)
:::
::::


## Baby Names

:::: {.columns}
::: {.column width="30%"}
Using a list of 8000 baby names from this [link](http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/), Karpathy trained an RNN to predict baby names.

To the right is a list of generated names not on the lists. More of the names can be found [here](https://cs.stanford.edu/people/karpathy/namesGenUnique.txt).

Some interesting additional names the model generated were
`Baby, Char, Mars, Hi, and With`.
:::
::: {.column width="70%"}
![](figs/BabyNames.png)
:::
::::


## Recap

We discussed the basic RNN architecture. We then discussed the LSTM and GRU modifications. These modifications allowed the RNNs to handle long-term dependencies.

We then considered an example application of energy consumption prediction.

We also discussed the unreasonable effectiveness of RNNs.

## What's Next?

As we saw, the RNN architecture has evolved to include LSTMs and GRUs. However, the RNN architecture also evolved to add attention. This was introduced by @bahdanau2014neural. 

Attention in language models is a mechanism that allows the model to focus on relevant parts of the input sequence by assigning different weights to different words, enabling it to capture long-range dependencies and context more effectively.

In addition, recall that RNNs are only able to process data sequentially. For large scale natural language processing applications, this is a major computational bottleneck. This motivated the development of more advanced neural network architectures that could process sequential data in parallel and utilize attention. 

The transformer architecture was introduced by @vaswani2017attention and combines both of these desirable features.

Transformers form the basis for modern large language models (LLMs) and we will discuss them in our NLP lectures.

## References

1. Understanding LSTMs, Colah’s blog, 2015, [https://colah.github.io/posts/2015-08-Understanding-LSTMs/](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) 
1. Speech and Language Processing. Daniel Jurafsky & James H. Martin. Draft of January 5, 2024. – Chapter 9, RNNs and LSTMs, [https://web.stanford.edu/~jurafsky/slpdraft/9.pdf](https://web.stanford.edu/~jurafsky/slpdraft/9.pdf) 
1. The Unreasonable Effectiveness of LSTMs, Andrej Karpathy, 2015, [https://karpathy.github.io/2015/05/21/rnn-effectiveness/](https://karpathy.github.io/2015/05/21/rnn-effectiveness/)